# Purpose

Demonstrate how to make SQL-based features testable and verifiable within Python client, as well as show how the same client can run in pre-production and production environments.

# Setup

To reproduce this, you'll need to:

1. Install dependencies: `pip install google-cloud-bigquery pyyaml jinja2`
1. Setup [GCP authenication](https://cloud.google.com/docs/authentication/getting-started)
1. Create a BigQuery table and populate using the national summary data from the [Atlantic COVID Tracking Project data](https://covidtracking.com/data/download).

# Sample data

Here's a sample query using the BigQuery client.

In [56]:
from google.cloud import bigquery

client = bigquery.Client()

QUERY = (
    """SELECT date, state, death, hospitalized 
         FROM `testable-features-poc.covid.us-states` 
        ORDER BY date DESC, state ASC
        LIMIT 5""")
query_job = client.query(QUERY)
rows = query_job.result()

for row in rows:
    print(row)

Row((datetime.date(2021, 3, 7), 'AK', 305, 1293), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AL', 10148, 45976), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AR', 5319, 14926), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AS', 0, None), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})
Row((datetime.date(2021, 3, 7), 'AZ', 16328, 57907), {'date': 0, 'state': 1, 'death': 2, 'hospitalized': 3})


## xxx
xxx

In [91]:
import yaml

feature_def = yaml.safe_load("""
  parameters:
    - foo: bar
  sources:
    - source1: testable-features-poc.covid.us-states
  query: |
    SELECT death
      FROM `testable-features-poc.covid.us-states`
     WHERE state = '{{ state }}'
     ORDER BY date DESC
     LIMIT 1
""")

In [96]:
from jinja2 import Template

class Connection:
    
    def __init__(self, feature):
        self.feature_def = feature
        self.gbq = bigquery.Client()
    
    def inference(self, **kwargs):
        t = Template(self.feature_def['query'])
        q = t.render(kwargs)
        #print(q)
        job = self.gbq.query(q)
        val = next(job.result(), None)
        return val[0] if val else None
    
    def close(self):
        print("closed")

class FeaturesClient:
        
    @classmethod
    def load_feature(cls, feature):
        return Connection(feature)

In [99]:
c = FeaturesClient.load_feature(feature_def)
r = c.inference(state="DC")
print(r)

1030
